In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import os
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn import metrics

pd.set_option('display.max_colwidth',100000) #https://stackoverflow.com/questions/54692405/output-truncation-in-google-colab
pd.set_option('display.max_rows', 100000)

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

In [ ]:
embeddings = pd.read_pickle('Downloads/embeddings.pkl')
embeddings = embeddings.drop(['image/format'], 
               axis=1)

print(len(embeddings['embedding']))
print(type(embeddings['embedding']))
print(type(embeddings['embedding'].iloc[2]))
print(embeddings['embedding'].iloc[2].size)
print(embeddings['Len'].nunique())

embeddings['dicom']= embeddings['image/id'].str[61:-4]

In [ ]:
array = embeddings['embedding'].tolist()
array_of_arrays= np.stack(array , axis=0)

In [ ]:
pca = PCA()



# Fit the pipeline to 'samples'
pca.fit(array_of_arrays)

# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_)
plt.xlabel('PCA feature')
plt.ylabel('variance')
plt.ylim(0,10)
plt.xticks(features)

plt.show()




In [ ]:
print(pca.explained_variance_.cumsum())
print(pca.explained_variance_ratio_.cumsum())

In [ ]:
cum_explained_var = []
for i in range(0, len(pca.explained_variance_ratio_)):
    if i == 0:
        cum_explained_var.append(pca.explained_variance_ratio_[i])
    else:
        cum_explained_var.append(pca.explained_variance_ratio_[i] + 
                                 cum_explained_var[i-1])

print(cum_explained_var)

In [ ]:
x= np.array(cum_explained_var) 
y= x < 0.8
print(len(np.where(y)[0]))

In [ ]:
# Import PCA
from sklearn.decomposition import PCA

# Create a PCA model with 2 components: pca
pca = PCA(n_components=62)

# Fit the PCA instance to the scaled samples
pca.fit(array_of_arrays)

# Transform the scaled samples: pca_features
pca_features = pca.transform(array_of_arrays)

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i, init = "k-means++", 
                   max_iter= 300, n_init=10)
    kmeans.fit(pca_features)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11), wcss)
plt.title("Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
# plt.savefig('Downloads/Elbow_method')
plt.show()

In [ ]:
# Import TSNE
from sklearn.manifold import TSNE

# Create a TSNE instance: model
model = TSNE(learning_rate=250, perplexity=120)

# Apply fit_transform to samples: tsne_features
tsne_features = model.fit_transform(pca_features)

# Select the 0th feature: xs
xs = tsne_features[:,0]

# Select the 1st feature: ys
ys = tsne_features[:,1]

In [ ]:
kmeans_labels450 = pd.read_pickle('Downloads/kmeans_labels450')
kmeans_labels450.head(5)

In [ ]:
kmeans_labels450['tsne_features']= kmeans_labels450[['tsne_features_xs', 'tsne_features_ys' ]].values.tolist()
kmeans_labels450.head(5)

In [ ]:
tsne_list = kmeans_labels450['tsne_features'].tolist()
tsne_array= np.stack(tsne_list , axis=0)

In [ ]:
tsne_features = tsne_array

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=6)
model.fit(tsne_features)
label = model.predict(tsne_features)

In [ ]:
print(f"Silhouette Coefficient: {metrics.silhouette_score(tsne_features, label):.3f}")

# The value of the silhouette coefﬁcient is between [-1, 1]. 
# A score of 1 denotes the best, meaning that the data point i is very compact within 
# the cluster to which it belongs and far away from the other clusters. 
# The worst value is -1. Values near 0 denote overlapping clusters

In [ ]:
xs = kmeans_labels450['tsne_features_xs']
ys = kmeans_labels450['tsne_features_ys']

In [ ]:
plt.scatter(xs,ys, marker ='.', alpha = 0.5, s=10, c=label)
plt.savefig('Downloads/kmeans_on_tsne_features_0.450.png', format='png', dpi=1200)
plt.show()

In [ ]:
# kmeans_labels_c6 = label

# kmeans_labels = pd.DataFrame({'embedding': embeddings['image/id'], 'label':label, 'dicom': embeddings['dicom'], 'tsne_features_xs':xs, 'tsne_features_ys':ys})

# kmeans_labels.to_pickle('Downloads/kmeans_labels_0.449')

In [ ]:
cluster_centers =[
 [-13.855849    -2.757724  ]
 [ 30.886284   -12.211306  ]
 [ -6.2821727   22.937733  ]
 [ 11.552219     6.9363875 ]
 [  0.77422804 -25.492647  ]
 [-31.335146    15.596095  ]]

In [ ]:
df = kmeans_labels450.copy()
df['c0'] = np.sqrt((df.tsne_features_xs - (-13.855849)) ** 2 + (df.tsne_features_ys - (-2.757724)) ** 2)
df['c1'] = np.sqrt((df.tsne_features_xs - (30.886284)) ** 2 + (df.tsne_features_ys - (-12.211306)) ** 2)
df['c2'] = np.sqrt((df.tsne_features_xs - (-6.2821727)) ** 2 + (df.tsne_features_ys - (22.937733)) ** 2)
df['c3'] = np.sqrt((df.tsne_features_xs - (11.552219)) ** 2 + (df.tsne_features_ys - (6.9363875)) ** 2)
df['c4'] = np.sqrt((df.tsne_features_xs - (0.77422804)) ** 2 + (df.tsne_features_ys - (-25.492647)) ** 2)
df['c5'] = np.sqrt((df.tsne_features_xs - (-31.335146)) ** 2 + (df.tsne_features_ys - (15.596095)) ** 2)

df.head(5)

In [ ]:
df0 = df.sort_values(by=['c0'])
df0.head(5)

In [ ]:
df1 = df.sort_values(by=['c1'])
df1.head(6)

In [ ]:
df2 = df.sort_values(by=['c2'])
df2.head(5)

In [ ]:
df3 = df.sort_values(by=['c3'])
df3.head(10)

In [ ]:
df4 = df.sort_values(by=['c4'])
df4.head(5)

In [ ]:
df5 = df.sort_values(by=['c5'])
df5.head(5)

In [ ]:
df5_icd_devices = df.sort_values(by=['tsne_features_ys'])
df5_icd_devices.head(10)

In [ ]:
hosp_mort = pd.read_csv('Downloads/hosp_mort.csv')
icu_los = pd.read_csv('Downloads/icu_los.csv')
final_adult_patients = pd.read_csv('Downloads/final_adult_patients.csv')
hosp_mort = hosp_mort.drop(['Unnamed: 0'], 
               axis=1)
hosp_mort.head(5)

In [ ]:
final_adult_patients.columns.values

In [ ]:
final_adult_patients = final_adult_patients.drop(['Unnamed: 0.1', 'Unnamed: 0','anchor_year', 'anchor_age'], 
               axis=1)
final_adult_patients = final_adult_patients.where(final_adult_patients['hadm_id_x'] == final_adult_patients['hadm_id_y'])
final_adult_patients = final_adult_patients[final_adult_patients['dicom'].notna()]

In [ ]:
print(len(final_adult_patients['dicom']))

In [ ]:
tro = pd.DataFrame({'subject_id': final_adult_patients['subject_id'],
                   'hadm_id': final_adult_patients['hadm_id_x'],
                   'dicom': final_adult_patients['dicom'], 'pO2_time':final_adult_patients['charttime_pO2'],
                    'admittime':final_adult_patients['admittime'], 'Xray_date':final_adult_patients['StudyDate'], 
                   'Xray_time': final_adult_patients['StudyTime'], 'pO2_date':final_adult_patients['pO2_date'],
                    'PF_ratio':final_adult_patients['PF_ratio'], 'charttime_pO2':final_adult_patients['charttime_pO2'] })
troy = tro.merge(hosp_mort, on='hadm_id', how = 'inner')
troy.head(10)

In [ ]:
print(len(troy['dicom']))

In [ ]:
troys = troy.merge(kmeans_labels450, on= "dicom", how='inner')
print(len(troys['dicom']))
troys = troys.drop(['image/id'], axis=1)
troys.head(100)

In [ ]:
print(len(troys['dicom']))
print(troys['dicom'].nunique())

In [ ]:
# duplicate = troys[troys.duplicated('dicom')]
# duplicate.head(20)

In [ ]:
troys = troys.drop_duplicates(subset='dicom', ignore_index=True)

In [ ]:
print(len(troys['dicom']))
print(troys['dicom'].nunique())

In [ ]:
troys.head(100)

In [ ]:
print(troys['subject_id_x'].nunique())
print(troys['hadm_id'].nunique())
print(troys['dicom'].nunique())

In [ ]:
base_table = troys.copy()
base_table = base_table.drop(['subject_id_y', 'hospital_expire_flag'], axis=1)
base_table.columns.values

In [ ]:
print(type(base_table))

In [ ]:
base_table = base_table.reindex(columns=['subject_id_x', 'hadm_id', 'dicom', 'PF_ratio','charttime_pO2', 'pO2_time', 'pO2_date', 'Xray_date',
                         'Xray_time', 'admittime', 'label', 'tsne_features_xs', 'tsne_features_ys', 'tsne_features'])
base_table.head(5)

In [ ]:
print(base_table['hadm_id'].nunique())
print(len(icu_los))

In [ ]:
# base_table.to_pickle('Downloads/base_table.pkl')

In [ ]:
ct = pd.crosstab(troys['label'], troys['hospital_expire_flag'], normalize= 'index')
ct.head(6)

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Create a contingency table of the counts of outcomes by cluster
contingency_table = pd.crosstab(troys['label'], troys['hospital_expire_flag'])

# Perform the chi-square test of independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square test statistic: {chi2:.3f}")
print(f"P-value: {p}")

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import multipletests

# assume df is your pandas dataframe with two columns: 'cluster_labels' and 'outcomes'
cont_table = pd.crosstab(troys['label'], troys['hospital_expire_flag'])

# Method 1: Bonferroni correction
alpha = 0.05
n_comparisons = len(cont_table)*(len(cont_table)-1)/2
alpha_adj = alpha/n_comparisons
reject = []
for i in range(len(cont_table)-1):
    for j in range(i+1, len(cont_table)):
        obs = np.array([cont_table.iloc[i,:], cont_table.iloc[j,:]])
        chi2, p, dof, exp = chi2_contingency(obs)
        reject.append(p < alpha_adj)
        
# Method 2: Pairwise chi-squared test
pvals = []
reject_pw = []
for i in range(len(cont_table)-1):
    for j in range(i+1, len(cont_table)):
        obs = np.array([cont_table.iloc[i,:], cont_table.iloc[j,:]])
        chi2, p, dof, exp = chi2_contingency(obs)
        pvals.append(p)
        reject_pw.append(p < alpha)

# Apply multiple comparisons correction to Method 2
reject_pw_corr = multipletests(pvals, alpha=alpha, method='bonferroni')[0]

# Print results
print("Method 1: Bonferroni correction")
print(f"Adjusted significance level: {alpha_adj}")
print(f"Reject null hypothesis for {sum(reject)} pairwise comparisons")
print(reject)

print("\nMethod 2: Pairwise chi-squared test")
for i in range(len(cont_table)-1):
    for j in range(i+1, len(cont_table)):
        print(f"Comparison {i+1}-{j+1}: p={pvals[i*(len(cont_table)-i-1)//2+j-i-1]}, reject={reject_pw_corr[i*(len(cont_table)-i-1)//2+j-i-1]}")
print(reject_pw_corr)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import statsmodels.stats.multicomp as mc



# compute the contingency table
ct = pd.crosstab(troys['label'], troys['hospital_expire_flag'])

# perform the chi-squared test
chi2, p, dof, expected = chi2_contingency(ct)

# print the chi-squared test results
print("Chi-Squared Test Results:")
print(f"Chi-Squared Statistic: {chi2:.3f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-value: {p:.3f}")

# perform the Tukey HSD test with Bonferroni correction
group_names = ['group1', 'group2', 'group3', 'group4', 'group5', 'group6']
tukey_result = mc.pairwise_tukeyhsd(troys['label'], troys['hospital_expire_flag'])
tukey_summary = tukey_result.summary()
print("\nTukey HSD Test Results (Bonferroni Corrected):")
print(tukey_summary)


In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Create a contingency table of observed frequencies
cont_table = pd.crosstab(troys['label'], troys['hospital_expire_flag'])

# Run the chi-squared test
chi2, p, dof, expected = chi2_contingency(cont_table)

# Print the results of the chi-squared test
print("Chi-squared test results:")
print(f"  Test statistic: {chi2:.4f}")
print(f"  Degrees of freedom: {dof}")
print(f"  p-value: {p:.4f}")

# Run the Tukey HSD test
tukey_results = pairwise_tukeyhsd(troys['label'], troys['hospital_expire_flag'])

# Print the pairwise comparisons with significant differences
group_names = cont_table.index
tukey_table = pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0])
tukey_table = tukey_table[['group1', 'group2', 'reject', 'p-adj']]
tukey_table = tukey_table[tukey_table['reject']]
tukey_table['group1'] = tukey_table['group1'].astype(int).apply(lambda x: group_names[x])
tukey_table['group2'] = tukey_table['group2'].astype(int).apply(lambda x: group_names[x])
print("Pairwise comparison results:")
print(tukey_table)


In [ ]:
print(group_names)

In [ ]:
from scipy import stats

x =[ 169.315183, 172.734661, 182.768118, 173.674676, 173.418298, 170.526781]
stats.shapiro(x)

In [ ]:
from scipy.stats import f_oneway
c1 = np.array([169.315183])
c2 = np.array([172.734661])
c3 = np.array([182.768118])
c4 = np.array([173.674676])
c5 = np.array([173.418298])
c6 = np.array([170.526781])

stats.f_oneway(c1,c2,c3,c4,c5,c6)

In [ ]:
from scipy.stats import chisquare
chisquare([0.169315183, 0.172734661, 0.182768118, 0.173674676, 0.173418298, 0.170526781])

In [ ]:
# ml = hosp_mort.merge(icu_los, how='inner', on='hadm_id')
# ml = ml.reset_index()
# ml.head(5)

In [ ]:
triy = pd.DataFrame({'subject_id': final_adult_patients['subject_id'],
                   'hadm_id': final_adult_patients['hadm_id_x'],
                   'dicom': final_adult_patients['dicom'],
                   'PF_ratio': final_adult_patients['PF_ratio']})

In [ ]:
triys = triy.merge(kmeans_labels450, on= "dicom", how='inner')
print(len(triys['dicom']))
triys = triys.drop(['image/id'], axis=1)

In [ ]:
triys = triys.drop_duplicates(subset='dicom', ignore_index=True)

In [ ]:
print(len(triys['dicom']))
print(triys['dicom'].nunique())

In [ ]:
mean = triys['PF_ratio'].groupby(triys['label']).mean()

In [ ]:
mean.head(6)

In [ ]:
stats.kstest(triys['PF_ratio'], 'norm')

In [ ]:
triys.columns.values

In [ ]:
PFc0 = triys['PF_ratio'].where(triys['label'] == 0)
PFc0 = PFc0.dropna()

PFc1 = triys['PF_ratio'].where(triys['label'] == 1)
PFc1 = PFc1.dropna()

PFc2 = triys['PF_ratio'].where(triys['label'] == 2)
PFc2 = PFc2.dropna()

PFc3 = triys['PF_ratio'].where(triys['label'] == 3)
PFc3 = PFc3.dropna()

PFc4 = triys['PF_ratio'].where(triys['label'] == 4)
PFc4 = PFc4.dropna()

PFc5 = triys['PF_ratio'].where(triys['label'] == 5)
PFc5 = PFc5.dropna()

In [ ]:
stats.f_oneway(PFc0, PFc1, PFc2, PFc3, PFc4, PFc5)

In [ ]:
from scipy.stats import f_oneway
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.graphics.gofplots import qqplot
import warnings
from IPython.display import display, Math, Latex, Markdown

comparison = MultiComparison(triys['PF_ratio'], triys['label'])
comparison_results = comparison.tukeyhsd()
comparison_results.summary()

In [ ]:
tra = pd.DataFrame({'subject_id': final_adult_patients['subject_id'],
                   'hadm_id': final_adult_patients['hadm_id_x'],
                   'dicom': final_adult_patients['dicom']})
tray = tro.merge(icu_los, on='hadm_id', how = 'inner')

In [ ]:
trays = tray.merge(kmeans_labels450, on= "dicom", how='inner')
print(len(trays['dicom']))
trays = trays.drop(['image/id'], axis=1)

In [ ]:
trays = trays.drop_duplicates(subset='dicom', ignore_index=True)

In [ ]:
print(len(trays['dicom']))
print(trays['dicom'].nunique())

In [ ]:
mean2 = trays['los'].groupby(trays['label']).mean()
mean2.head(6)

In [ ]:
stats.kstest(trays['los'], 'norm')

In [ ]:
LOSc0 = trays['los'].where(trays['label'] == 0)
LOSc0 = LOSc0.dropna()

LOSc1 = trays['los'].where(trays['label'] == 1)
LOSc1 = LOSc1.dropna()

LOSc2 = trays['los'].where(trays['label'] == 2)
LOSc2 = LOSc2.dropna()

LOSc3 = trays['los'].where(trays['label'] == 3)
LOSc3 = LOSc3.dropna()

LOSc4 = trays['los'].where(trays['label'] == 4)
LOSc4 = LOSc4.dropna()

LOSc5 = trays['los'].where(trays['label'] == 5)
LOSc5 = LOSc5.dropna()

In [ ]:
stats.kruskal(LOSc0, LOSc1, LOSc2, LOSc3, LOSc4, LOSc5)

In [ ]:
comparison = MultiComparison(trays['los'], trays['label'])
comparison_results = comparison.tukeyhsd()
comparison_results.summary()